In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [0]:

## Load dataset
df = pd.read_csv('order_brush_order.csv')

In [0]:
df.head()

In [0]:
df.describe()

In [0]:
df['shopid'].nunique()

In [0]:
df['userid'].nunique()

In [0]:
df.groupby('shopid')['orderid'].count().mean()

In [0]:
## Minimum time 
df[df['event_time'] == df['event_time'].min()]

In [0]:
## Set event_time as index 
df_time = df.set_index(pd.DatetimeIndex(df['event_time'])).drop('event_time', axis=1)
df_time = df_time.sort_index()

## Groupby shopid, userid, and event_time 60 min intervals, count order(s)
grouped_orders = df_time.groupby(['shopid', 'userid', pd.Grouper(freq='60min', label='left', base=0)]).count()

In [0]:
## Find shopid that possibly do order brushing, given a threshold
possible_brush = grouped_orders[grouped_orders.orderid > 2]

userids = []
possible_brush.reset_index().groupby('shopid')['userid'].apply(lambda x: userids.append(x.values))
len(userids)

In [0]:
possible_brush.reset_index().shopid.nunique()

In [0]:
def concat_and(arr):
    res = '&'.join(str(x) for x in arr)
    return res

concat_userids = []
for element in userids:
    concat_userids.append(concat_and(element))

res_df = pd.DataFrame({"shopid": possible_brush.reset_index()['shopid'].unique(), "userid": concat_userids})
res_df.head()

In [0]:
## Final answer
default_df = pd.DataFrame({'shopid': df['shopid'].unique(), 'userid': 0})
default_df.head()

In [0]:
res_df.head()

In [0]:
res_df.shape

In [0]:
## Export Result
sol_df = pd.concat([default_df[~default_df.shopid.isin(res_df.shopid)], res_df])
sol_df.to_csv("solution_3.csv", index=False)

In [0]:
## Check result
pd.read_csv("solution_3.csv")